In [1]:
import pandas as pd
import Revisions
from time import time
import numpy as np
import re
from datetime import date, timedelta
from os.path import isfile
import pymysql

In [2]:
df_ep = pd.read_csv('070916_edit_protected_articles.csv')
df_nep = pd.read_csv('071016_non_edit_protected_articles.csv')

In [3]:
t0 = time()
start = 0
end = len(df_ep['page_id'].values)
revs = [Revisions.GetRevisions(df_ep['page_id'].values[i]) for i in range(start, end)]
print(time() - t0)

2948.918939113617


In [22]:
t0 = time()
dfs = [Revisions.RevisionToWeeklyAgg(revs[i]) if revs[i] is not None else None for i in range(start, end)]
print(time() - t0)

467.97453117370605


In [23]:
path1 = 'RevisionsOverTime.csv'
path2 = 'Proections.csv'
for item in dfs:
    if item is not None:
        header = not isfile(path1)
        item[0].to_csv(path1, mode='a', header=header, index=False)
        item[1].to_csv(path2, mode='a', header=header, index=False)

In [24]:
ls -lh *.csv

-rw-r--r--  1 dekedor  staff   1.1M Jul 10 10:43 070916_edit_protected_articles.csv
-rw-r--r--  1 dekedor  staff   5.2M Jul 10 23:39 071016_non_edit_protected_articles.csv
-rw-r--r--  1 dekedor  staff   7.2M Jul 14 12:08 Proections.csv
-rw-r--r--  1 dekedor  staff    32M Jul 14 12:08 RevisionsOverTime.csv
-rw-r--r--  1 dekedor  staff   261K Jul  8 19:56 candidate_titles_edit_locked_articles.csv
